### Setting up a python REST API

* REST (REpresentational State Transfer) 
* the standard architectural design for web services and web APIs.

<table class="data">
<tr><th>HTTP Method</th><th>Action</th><th>Examples</th></tr>
<tr><td>GET</td><td>Obtain information about a resource</td><td>http://example.com/api/orders<br />(retrieve order list)</td></tr>
<tr><td>GET</td><td>Obtain information about a resource</td><td>http://example.com/api/orders/123<br />(retrieve order #123)</td></tr>
<tr><td>POST</td><td>Create a new resource</td><td>http://example.com/api/orders<br />(create a new order, from data provided with the request)</td></tr>
<tr><td>PUT</td><td>Update a resource</td><td>http://example.com/api/orders/123<br />(update order #123, from data provided with the request)</td></tr>
<tr><td>DELETE</td><td>Delete a resource</td><td>http://example.com/api/orders/123<br />(delete order #123)</td></tr>
</table>


#### Installing Flask
```shell
$ mkdir todo-api
$ cd todo-api
$ virtualenv flask_env

$ flask_env/bin/pip install flask
```

#### app.py
```python
#!flask-env/bin/python
from flask import Flask

app = Flask(__name__)

@app.route('/')
def index():
    return "Hello, World!"

if __name__ == '__main__':
    app.run(debug=True)
```

####Starting with GET 

```python
#!flask-env/bin/python
#REST_app_get.py
from flask import Flask, jsonify

app = Flask(__name__)

tasks = [
    {
        'id': 1,
        'title': u'Learn Python',
        'description': u'import this', 
        'done': False
    },
    {
        'id': 2,
        'title': u'Learn REST',
        'description': u'Need to lears REST to setup APIs for automation framework', 
        'done': False
    }
]

@app.route('/todo/api/v1.0/tasks', methods=['GET'])
def get_tasks():
    return jsonify({'tasks': tasks})

if __name__ == '__main__':
    app.run(debug=True)
```

In [5]:
!curl -i http://localhost:5000/todo/api/v1.0/tasks







{
  "tasks": [
    {
      "description": "import this",
      "done": false,
      "id": 1,
      "title": "Learn Python"
    },
    {
      "description": "Need to learn REST to setup APIs for automation framework",
      "done": false,
      "id": 2,
      "title": "Learn REST"
    }
  ]
}

#### Adding a POST method:
```python
@app.route('/todo/api/v1.0/tasks', methods=['POST'])
def create_task():
    if not request.json or not 'title' in request.json:
        abort(400)
    task = {
        'id': tasks[-1]['id'] + 1,
        'title': request.json['title'],
        'description': request.json.get('description', ""),
        'done': False
    }
    tasks.append(task)
    return jsonify({'task': task}), 201
```

In [7]:
!curl -i -H "Content-Type: application/json" -X POST -d '{"title":"Read a book"}' http://localhost:5000/todo/api/v1.0/tasks







{
  "task": {
    "description": "",
    "done": false,
    "id": 3,
    "title": "Read a book"
  }
}

#### PUT and DELETE
```python
@app.route('/todo/api/v1.0/tasks/<int:task_id>', methods=['PUT'])
def update_task(task_id):
    task = [task for task in tasks if task['id'] == task_id]
    if len(task) == 0:
        abort(404)
    if not request.json:
        abort(400)
    if 'title' in request.json and type(request.json['title']) != unicode:
        abort(400)
    if 'description' in request.json and type(request.json['description']) is not unicode:
        abort(400)
    if 'done' in request.json and type(request.json['done']) is not bool:
        abort(400)
    task[0]['title'] = request.json.get('title', task[0]['title'])
    task[0]['description'] = request.json.get('description', task[0]['description'])
    task[0]['done'] = request.json.get('done', task[0]['done'])
    return jsonify({'task': task[0]})

@app.route('/todo/api/v1.0/tasks/<int:task_id>', methods=['DELETE'])
def delete_task(task_id):
    task = [task for task in tasks if task['id'] == task_id]
    if len(task) == 0:
        abort(404)
    tasks.remove(task[0])
    return jsonify({'result': True})
```

In [8]:
#Updates a record
! curl -i -H "Content-Type: application/json" -X PUT -d '{"done":true}' http://localhost:5000/todo/api/v1.0/tasks/2








{
  "task": {
    "description": "Need to learn REST to setup APIs for automation framework",
    "done": true,
    "id": 2,
    "title": "Learn REST"
  }
}